In [8]:
!pip install langchain faiss-cpu pypdf sentence-transformers
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00


In [9]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

In [ ]:
# 1. Carregar PDFs
loader = PyPDFLoader("stride.pdf")
pages = loader.load()

# Geração de Relatório STRIDE com RAG e Salvamento no Google Drive (Colab)
Este notebook executa um pipeline de geração de relatório STRIDE usando RAG (busca contextual) e LLM (phi-2), com opção de salvar o resultado no Google Drive via Colab.

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import faiss
import numpy as np
import json

# Para integração com Google Drive no Colab
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

In [ ]:
# Base de conhecimento simulada
DOCUMENTOS = [
    {"componente": "Amazon API Gateway", "texto": "API Gateway pode ser explorado por spoofing se não houver autenticação adequada nas rotas de entrada."},
    {"componente": "AWS Lambda", "texto": "Lambda pode ter risco de escalonamento de privilégio se permissões IAM estiverem amplas."},
    {"componente": "Amazon S3", "texto": "S3 frequentemente sofre com divulgação indevida de dados quando buckets ficam publicamente acessíveis."},
    {"componente": "IAM", "texto": "IAM mal configurado é um vetor crítico de spoofing e privilege escalation."}
]

In [ ]:
# Gerar embeddings e indexar com FAISS
model_emb = SentenceTransformer('all-MiniLM-L6-v2')
frases = [doc['texto'] for doc in DOCUMENTOS]
embeddings = model_emb.encode(frases, convert_to_numpy=True)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [ ]:
# Função de busca contextual (RAG)
def recuperar_contexto(componentes, top_k=3):
    contexto = []
    for componente in componentes:
        consultas = [doc['texto'] for doc in DOCUMENTOS if doc['componente'] == componente]
        if not consultas:
            continue
        emb = model_emb.encode(consultas[0], convert_to_numpy=True).reshape(1, -1)
        dists, idxs = index.search(emb, top_k)
        for i in idxs[0]:
            contexto.append(frases[i])
    return "\n".join(set(contexto))

In [ ]:
# Função de geração de relatório com LLM (phi-2)
def gerar_relatorio(componentes, contexto):
    prompt = f"""
Você é um analista de segurança da informação. Com base nos componentes abaixo e no conteúdo técnico a seguir, gere um relatório explicando:\n1. Ameaças detectadas (por STRIDE)\n2. Justificativa técnica\n3. Recomendações\n\nComponentes:\n{', '.join(componentes)}\n\nContexto técnico:\n{contexto}\n\nRelatório:\n"""
    tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
    model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    resposta = pipe(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)[0]['generated_text']
    return resposta

In [ ]:
# Executar pipeline de geração de relatório
componentes_detectados = ["Amazon API Gateway", "AWS Lambda", "Amazon S3"]  # Edite conforme necessário
contexto = recuperar_contexto(componentes_detectados)
relatorio = gerar_relatorio(componentes_detectados, contexto)
print("\n===== RELATÓRIO GERADO =====\n")
print(relatorio)

In [ ]:
# Salvar relatório gerado em caminho definido pelo usuário (Google Drive)
if IN_COLAB:
    drive.mount('/content/drive')
    save_path = input('Digite o caminho completo para salvar o relatório (ex: /content/drive/MyDrive/relatorio_stride.txt): ')
else:
    save_path = input('Digite o caminho completo para salvar o relatório (ex: ./relatorio_stride.txt): ')

with open(save_path, 'w', encoding='utf-8') as f:
    f.write(relatorio)
print(f'Relatório salvo em: {save_path}')

In [ ]:
# Instale as dependências necessárias (execute apenas uma vez por ambiente)
!pip install sentence-transformers faiss-cpu transformers

In [ ]:
!pip install -U langchain-community